<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/Nov%C3%BD_jazykov%C3%BD_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nový jazykový model**



# Krok 1: Příprava prostředí Google Colab

In [1]:
# Instalace PyTorch
!pip install torch torchvision
# Instalace transformerů
!pip install transformers
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, RandomSampler
from transformers import BertForSequenceClassification, AdamW
from torch.nn import CrossEntropyLoss
import random


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
#Připojení Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Krok 2: Tokenizace

In [3]:
# použití tokenizéru

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased') #  Model cased rozlišuje velká a malá písmena
"""
Pro specifické úkoly: Některé modely mohou být optimalizovány pro konkrétní úkoly,
jako je otázka a odpověď, sentimentální analýza atd. V takovém případě vyber model,
který nejlépe odpovídá...
"""

# Zde se načte text pro trénování:
!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/100x%20kytice.txt
with open('100x kytice.txt', 'r', encoding='utf-8') as f:
    text = f.read() # načte textový soubor k učení

# Generování náhodných štítků pro příklady
labels = [random.randint(0, 1) for _ in text]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

--2024-04-17 11:32:18--  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/KYTICE/10x%20KYTICE/100x%20kytice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8069800 (7.7M) [text/plain]
Saving to: ‘100x kytice.txt’

100x kytice.txt     100%[===================>]   7.70M  --.-KB/s    in 0.08s   

2024-04-17 11:32:19 (90.7 MB/s) - ‘100x kytice.txt’ saved [8069800/8069800]



# Krok 3: Příprava dat

In [4]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels  # Přidání štítků
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]  # Získání štítku
        inputs = self.tokenizer(text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors="pt")
        return inputs.input_ids.squeeze(), inputs.attention_mask.squeeze(), torch.tensor(label, dtype=torch.long)

dataset = CustomDataset(texts=text, labels=labels, tokenizer=tokenizer, max_length=512)


# Krok 4: Vytvoření modelu

BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2): Načte model BERT, který byl předtrénován na mnohojazyčných datech. Parametr num_labels=2 indikuje, že model bude použit pro úlohu klasifikace s dvěma třídami (např. pozitivní/negativní).

AdamW(model.parameters(), lr=5e-5): Inicializuje AdamW optimizer, který se často používá pro trénování modelů založených na Transformeru. Parametr lr udává rychlost učení.

In [5]:
# Předpokládáme, že `dataset` a `tokenizer` jsou již definovány
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Přesun modelu na GPU, pokud je dostupné
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

# Krok 5: Trénování modelu

Trénovací smyčka

In [7]:

train_dataloader = DataLoader(dataset, sampler=RandomSampler(dataset), batch_size=4)

# Definice ztrátové funkce
loss_fn = CrossEntropyLoss()

# Nastavení modelu do trénovacího režimu
model.train()

# Trénovací smyčka
for epoch in range(4):  # Počet epoch
    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        # Přesun dat na stejné zařízení jako model
        b_input_ids = b_input_ids.to(model.device)
        b_input_mask = b_input_mask.to(model.device)
        b_labels = b_labels.to(model.device)

        optimizer.zero_grad()  # Vynulování gradientů modelu

        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss  # Získání loss z výstupů modelu

        loss.backward()  # Backpropagation
        optimizer.step()  # Aktualizace váh modelu

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


Epoch: 0, Loss: 0.7415187954902649
Epoch: 0, Loss: 0.7518604397773743
Epoch: 0, Loss: 0.6533899307250977
Epoch: 0, Loss: 0.7015507221221924
Epoch: 0, Loss: 0.7322866916656494
Epoch: 0, Loss: 0.6767868995666504
Epoch: 0, Loss: 0.6423875689506531
Epoch: 0, Loss: 0.7057780027389526
Epoch: 0, Loss: 0.7019451260566711
Epoch: 0, Loss: 0.7273581027984619
Epoch: 0, Loss: 0.7045090198516846
Epoch: 0, Loss: 0.7176315188407898
Epoch: 0, Loss: 0.6845593452453613
Epoch: 0, Loss: 0.6826408505439758
Epoch: 0, Loss: 0.6781123876571655
Epoch: 0, Loss: 0.7618603706359863
Epoch: 0, Loss: 0.7132947444915771
Epoch: 0, Loss: 0.6877936720848083
Epoch: 0, Loss: 0.6921654343605042
Epoch: 0, Loss: 0.649250328540802
Epoch: 0, Loss: 0.5059864521026611
Epoch: 0, Loss: 0.5793402194976807
Epoch: 0, Loss: 0.6189966201782227
Epoch: 0, Loss: 0.920063853263855
Epoch: 0, Loss: 0.7074836492538452
Epoch: 0, Loss: 0.581731915473938
Epoch: 0, Loss: 0.5957587957382202
Epoch: 0, Loss: 0.7434251308441162
Epoch: 0, Loss: 0.61183

KeyboardInterrupt: 